In [ ]:
# Requirements
!pip install transformers
!pip install openai
!pip install openTSNE
!pip install datasets

In [2]:
!git clone https://github.com/LC1332/Luotuo-Text-Embedding.git

Cloning into 'Luotuo-Text-Embedding'...
remote: Enumerating objects: 436, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 436 (delta 21), reused 32 (delta 10), pack-reused 388
Receiving objects: 100% (436/436), 29.68 MiB | 9.78 MiB/s, done.
Resolving deltas: 100% (248/248), done.


In [ ]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

In [15]:
%cd Luotuo-Text-Embedding

/content/Luotuo-Text-Embedding


In [16]:
import csv
import numpy as np
import sys
sys.path.append("..")
def get_evalCSV():
    text_left = []
    text_right = []
    with open("./data/sentspair.csv", "r") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            text_left.append(row[0])
            text_right.append(row[1])
    return text_left, text_right

text_left, text_right = get_evalCSV()
inputs = tokenizer(text_left, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_left = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
inputs = tokenizer(text_right, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_right = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
    
cos_sim_matrix = torch.matmul(embeddings_left, embeddings_right.t())
cos_sim_matrix /= torch.matmul(torch.norm(embeddings_left, dim=1, keepdim=True), torch.norm(embeddings_right, dim=1, keepdim=True).t())
tensor_cpu = cos_sim_matrix.cpu()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
from lib.tsne import TSNE_Plot

merged_list = text_left + text_right
merged_embed = torch.cat((embeddings_left, embeddings_right), dim=0)

# if the data have no labels, you can use the following code to cluster the data
tsne_plot = TSNE_Plot(merged_list, merged_embed, n_clusters = 4)
tsne_plot.tsne_plot(n_sentence=40)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [18]:
import pandas as pd
import lib.heatmap
import importlib
importlib.reload(lib.heatmap)

from lib.heatmap import Heatmap
# positions = [(i, i) for i in range(0, 20, 2)] + [(1, 5), (2, 3), (15, 9), (5, 13), (17, 7)]
# heatmap = Heatmap(df, positions)
df = pd.DataFrame({ "first":text_left, 
                    "second":text_right, 
                    "first_embed":[np.array(embeddings_left[i]) for i in range(len(embeddings_left))], 
                    "second_embed":[np.array(embeddings_right[i]) for i in range(len(embeddings_right))]})
heatmap = Heatmap(df)
heatmap.create_heatmap(font_path='./lib/arial.ttf')

# TODO:
* 模糊问题搜索
* 文本聚类
* 少样本分类学习